In [13]:
from omero.gateway import BlitzGateway
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics
import pandas as pd
from scipy import ndimage as ndi
from skimage import measure, exposure
from skimage.segmentation import clear_border
from skimage.morphology import remove_small_objects

import cv2

In [14]:
conn = BlitzGateway('hy274', 'omeroreset', host='ome2.hpc.susx.ac.uk')
conn.connect()

True

In [15]:
well_id=[6633,6592,6579,6557,6621,6580,6548,6565,6601,6587,6583,6591,6622,6551,6631,6575,6558,6620,6576,6555,6614,6753,6596,6599,6603,6636,6588,6600,6626,6608,6641,6638,6612,6607,6532,6531,6454,6461,6470,6459,6477,6461,6536,6525,6523,6462,6504,6505,6468,6510,6526,6545,6466,6544,6509,6499,6457,6460,6487,6476,6494,6488,6518,6530,
         6452,6476,6481,6483,6511,6473,6456,6486,6506,6451,6533,6497,6517,6512,6521,6500,6502,6488,6542,6475,6527,6490,6515,6516,6453,6465,6471,6507,6491,6528,6492,6531,6496,6484,6539,6478,6534,6513,6451,6493,6540,6543,6480,6472,6514,6482,6538,6498,6503,6485,6524,6535,6546,6479,6522,6469,6495,6529,6519,6508,6520,6474,
         6463,6455,6501,6458]

In [16]:
def overlay_mask(dapi_mask,cellmask):
    n_masks=dapi_mask
    masks=cellmask
    overlap=(masks!=0) * (n_masks!=0)
    list_n_masks = np.stack([n_masks[overlap], masks[overlap]])[-2].tolist()
    list_masks = np.stack([n_masks[overlap], masks[overlap]])[-1].tolist()
    overlay_all = {list_n_masks[i]: list_masks[i] for i in range(len(list_n_masks))}
    df = pd.DataFrame(list(overlay_all.items()), columns = ['Nuclei_ID','Cyto_ID'])
    return df

In [17]:
def dna_norm(df):
    """
    normalise histogram of DNA label
    :param df: input dataframe
    :return: input dataframe with additional column: "DNA_content"
    """
    y, x, _ = plt.hist(df['integrated_int_DAPI'], bins=10000)
    max = x[np.where(y == y.max())]
    df['DNA_content'] = df['integrated_int_DAPI'] / max[0]
    return df

In [18]:
def g1_list(df_all):
    return df_all.loc[(df_all['mean_intensity_Edu'] < 1000) & (df_all['DNA_content'] < 1.5),'Nuclei_ID'].tolist()
def s_list(df_all):
    return df_all.loc[df_all['mean_intensity_Edu'] > 1000,'Nuclei_ID'].tolist()
def g2_list(df_all):
    return df_all.loc[(df_all['mean_intensity_Edu'] < 1000) & (df_all['DNA_content'] > 1.5)& (df_all['mean_intensity_H3'] <20000) ,'Nuclei_ID'].tolist()
def m_list(df_all):
    return df_all.loc[(df_all['mean_intensity_H3'] >20000) & (df_all['DNA_content'] > 1.5) & (df_all['mean_intensity_Edu'] < 1000),'Nuclei_ID'].tolist()
def Cell_list(df_all,list):
    Cell_list=[]
    for i in list:
        Cell_list.append(df_all.loc[df_all['Nuclei_ID']==i ,'Cyto_ID'].values[0])
    return Cell_list

In [19]:
def save_data_label_g1(list_cyto,list_nuclei,ID):

    width=30
    if len(list_cyto)==len(list_nuclei):
        for index in range(0,len(list_cyto)):
            segmented_1=(masks*(masks==list_cyto[index]))
            segmented_2=(n_masks*(n_masks==list_nuclei[index]))
            if len(measure.regionprops(segmented_1))!=0 and len(measure.regionprops(segmented_2))!=0:
                region_1=measure.regionprops(segmented_1)[0]
                region_2=measure.regionprops(segmented_2)[0]
                b_1=imgCell[region_1.bbox[0]:region_1.bbox[2],region_1.bbox[1]:region_1.bbox[3]]
                b_2=imgDAPI[region_2.bbox[0]:region_2.bbox[2],region_2.bbox[1]:region_2.bbox[3]]
    # a=(masks*(masks==id))[region.bbox[0]:region.bbox[2],region.bbox[1]:region.bbox[3]]
    # c=(a!=0)*b
    # plt.imshow(a)
    # plt.show()
                if (10 <= len(b_1) <= 60) and (10 <= len(b_2) <= 60):
                    centroid=region_1.centroid


    # plt.imshow(b)
    # plt.show()

                    i = centroid[0]
                    j = centroid[1]
                    imin = int(round(max(0, i-width)))
                    imax = int(round(min(masks.shape[0], i+width+1)))
                    jmin = int(round(max(0, j-width)))
                    jmax = int(round(min(masks.shape[1], j+width+1)))
                    box_mask_1=segmented_1[imin:imax, jmin:jmax]
                    box_img_1=imgCell[imin:imax, jmin:jmax]
                    box_1=(box_mask_1!=0)*box_img_1

                    box_mask_2=segmented_2[imin:imax, jmin:jmax]
                    box_img_2=imgDAPI[imin:imax, jmin:jmax]
                    box_2=(box_mask_2!=0)*box_img_2
                    if box_1.shape[0]==box_1.shape[1] and box_2.shape[0]==box_2.shape[1]:
                        cv2.imwrite('/Users/haoranyue/Documents/master_project/Output3/'+str(int(ID))+'_G1_'+str(int(list_cyto[index]))+'.tif',box_1)
                        cv2.imwrite('/Users/haoranyue/Documents/master_project/Output4/' +str(int(ID))+'_G1_'+str(int(list_cyto[index]))+'.tif',box_2)

In [20]:
def save_data_label_s(list_cyto,list_nuclei,ID):

    width=30
    if len(list_cyto)==len(list_nuclei):
        for index in range(0,len(list_cyto)):
            segmented_1=(masks*(masks==list_cyto[index]))
            segmented_2=(n_masks*(n_masks==list_nuclei[index]))
            if len(measure.regionprops(segmented_1))!=0 and len(measure.regionprops(segmented_2))!=0:
                region_1=measure.regionprops(segmented_1)[0]
                region_2=measure.regionprops(segmented_2)[0]
                b_1=imgCell[region_1.bbox[0]:region_1.bbox[2],region_1.bbox[1]:region_1.bbox[3]]
                b_2=imgDAPI[region_2.bbox[0]:region_2.bbox[2],region_2.bbox[1]:region_2.bbox[3]]
    # a=(masks*(masks==id))[region.bbox[0]:region.bbox[2],region.bbox[1]:region.bbox[3]]
    # c=(a!=0)*b
    # plt.imshow(a)
    # plt.show()
                if (10 <= len(b_1) <= 60) and (10 <= len(b_2) <= 60):
                    centroid=region_1.centroid


    # plt.imshow(b)
    # plt.show()

                    i = centroid[0]
                    j = centroid[1]
                    imin = int(round(max(0, i-width)))
                    imax = int(round(min(masks.shape[0], i+width+1)))
                    jmin = int(round(max(0, j-width)))
                    jmax = int(round(min(masks.shape[1], j+width+1)))
                    box_mask_1=segmented_1[imin:imax, jmin:jmax]
                    box_img_1=imgCell[imin:imax, jmin:jmax]
                    box_1=(box_mask_1!=0)*box_img_1

                    box_mask_2=segmented_2[imin:imax, jmin:jmax]
                    box_img_2=imgDAPI[imin:imax, jmin:jmax]
                    box_2=(box_mask_2!=0)*box_img_2
                    if box_1.shape[0]==box_1.shape[1] and box_2.shape[0]==box_2.shape[1]:
                        cv2.imwrite('/Users/haoranyue/Documents/master_project/Output3/'+str(int(ID))+'_S_'+str(int(list_cyto[index]))+'.tif',box_1)
                        cv2.imwrite('/Users/haoranyue/Documents/master_project/Output4/' +str(int(ID))+'_S_'+str(int(list_cyto[index]))+'.tif',box_2)

In [21]:
def save_data_label_g2(list_cyto,list_nuclei,ID):

    width=30
    if len(list_cyto)==len(list_nuclei):
        for index in range(0,len(list_cyto)):
            segmented_1=(masks*(masks==list_cyto[index]))
            segmented_2=(n_masks*(n_masks==list_nuclei[index]))
            if len(measure.regionprops(segmented_1))!=0 and len(measure.regionprops(segmented_2))!=0:
                region_1=measure.regionprops(segmented_1)[0]
                region_2=measure.regionprops(segmented_2)[0]
                b_1=imgCell[region_1.bbox[0]:region_1.bbox[2],region_1.bbox[1]:region_1.bbox[3]]
                b_2=imgDAPI[region_2.bbox[0]:region_2.bbox[2],region_2.bbox[1]:region_2.bbox[3]]
    # a=(masks*(masks==id))[region.bbox[0]:region.bbox[2],region.bbox[1]:region.bbox[3]]
    # c=(a!=0)*b
    # plt.imshow(a)
    # plt.show()
                if (10 <= len(b_1) <= 60) and (10 <= len(b_2) <= 60):
                    centroid=region_1.centroid


    # plt.imshow(b)
    # plt.show()

                    i = centroid[0]
                    j = centroid[1]
                    imin = int(round(max(0, i-width)))
                    imax = int(round(min(masks.shape[0], i+width+1)))
                    jmin = int(round(max(0, j-width)))
                    jmax = int(round(min(masks.shape[1], j+width+1)))
                    box_mask_1=segmented_1[imin:imax, jmin:jmax]
                    box_img_1=imgCell[imin:imax, jmin:jmax]
                    box_1=(box_mask_1!=0)*box_img_1

                    box_mask_2=segmented_2[imin:imax, jmin:jmax]
                    box_img_2=imgDAPI[imin:imax, jmin:jmax]
                    box_2=(box_mask_2!=0)*box_img_2
                    if box_1.shape[0]==box_1.shape[1] and box_2.shape[0]==box_2.shape[1]:
                        cv2.imwrite('/Users/haoranyue/Documents/master_project/Output3/'+str(int(ID))+'_G2_'+str(int(list_cyto[index]))+'.tif',box_1)
                        cv2.imwrite('/Users/haoranyue/Documents/master_project/Output4/' +str(int(ID))+'_G2_'+str(int(list_cyto[index]))+'.tif',box_2)

In [22]:
def save_data_label_m(list_cyto,list_nuclei,ID):

    width=30
    if len(list_cyto)==len(list_nuclei):
        for index in range(0,len(list_cyto)):
            segmented_1=(masks*(masks==list_cyto[index]))
            segmented_2=(n_masks*(n_masks==list_nuclei[index]))
            if len(measure.regionprops(segmented_1))!=0 and len(measure.regionprops(segmented_2))!=0:
                region_1=measure.regionprops(segmented_1)[0]
                region_2=measure.regionprops(segmented_2)[0]
                b_1=imgCell[region_1.bbox[0]:region_1.bbox[2],region_1.bbox[1]:region_1.bbox[3]]
                b_2=imgDAPI[region_2.bbox[0]:region_2.bbox[2],region_2.bbox[1]:region_2.bbox[3]]
    # a=(masks*(masks==id))[region.bbox[0]:region.bbox[2],region.bbox[1]:region.bbox[3]]
    # c=(a!=0)*b
    # plt.imshow(a)
    # plt.show()
                if (10 <= len(b_1) <= 60) and (10 <= len(b_2) <= 60):
                    centroid=region_1.centroid


    # plt.imshow(b)
    # plt.show()

                    i = centroid[0]
                    j = centroid[1]
                    imin = int(round(max(0, i-width)))
                    imax = int(round(min(masks.shape[0], i+width+1)))
                    jmin = int(round(max(0, j-width)))
                    jmax = int(round(min(masks.shape[1], j+width+1)))
                    box_mask_1=segmented_1[imin:imax, jmin:jmax]
                    box_img_1=imgCell[imin:imax, jmin:jmax]
                    box_1=(box_mask_1!=0)*box_img_1

                    box_mask_2=segmented_2[imin:imax, jmin:jmax]
                    box_img_2=imgDAPI[imin:imax, jmin:jmax]
                    box_2=(box_mask_2!=0)*box_img_2
                    if box_1.shape[0]==box_1.shape[1] and box_2.shape[0]==box_2.shape[1]:
                        cv2.imwrite('/Users/haoranyue/Documents/master_project/Output3/'+str(int(ID))+'_M_'+str(int(list_cyto[index]))+'.tif',box_1)
                        cv2.imwrite('/Users/haoranyue/Documents/master_project/Output4/' +str(int(ID))+'_M_'+str(int(list_cyto[index]))+'.tif',box_2)


In [23]:
well_id=[6513,6451,6493,6540,6543,6480,6472,6514,6482,6538,6498,6503,6485,6524,6535,6546,6479,6522,6469,6495,6529,6519,6508,6520,6474,
         6463,6455,6501,6458]
well_id[24]

6474

In [ ]:
from numpy import asarray
from PIL import Image
import pandas as pd
Feature_list=['label','area','max_intensity','mean_intensity']
# plateId=813
df_cell_phase=pd.DataFrame()
df_label=pd.DataFrame()
well_id=[6633,6592,6579,6557,6621,6580,6548,6565,6601,6587,6583,6591,6622,6551,6631,6575,6558,6620,6576,6555,6614,6753,6596,6599,6603,6636,6588,6600,6626,6608,6641,6638,6612,6607,6532,6531,6454,6461,6470,6459,6477,6461,6536,6525,6523,6462,6504,6505,6468,6510,6526,6545,6466,6544,6509,6499,6457,6460,6487,6476,6494,6488,6518,6530]

for ic,ind in enumerate(well_id):
    print(ic)
    WellID = ind
    well = conn.getObject("Well", WellID)
    df_final=pd.DataFrame()
    for index, image in enumerate(well.listChildren()):
        image = well.getImage(index)
        id_img=well.getImage(index).getId()
        pixels=image.getPrimaryPixels()
        imgall=image.renderImage(image.getSizeZ()/2,0)
        img_array=np.array(imgall)
        imgDAPI=pixels.getPlane(0,0,0)
        imgCell=pixels.getPlane(0,3,0)
        imgEdu=pixels.getPlane(0,1,0)
        imgH3=pixels.getPlane(0,2,0)
        img=np.stack((imgDAPI,imgCell,imgEdu,imgH3),axis=2)
        print(img.shape)



        #  #modle
        cyto2_tubulin_model_path='/Users/haoranyue/Documents/master_project/parpers/RPE-1_Tub_Hoechst'
        model = models.CellposeModel(gpu=False, pretrained_model=cyto2_tubulin_model_path)
        channels = [2,3]
        masks, flows, styles = model.eval(img_array, diameter=30.8, channels=channels)
        nuclei_DAPI_model_path='/Users/haoranyue/PycharmProjects/pythonProject/DPAI_813/models/nuclei_DAPI'
        n_model = models.CellposeModel(gpu=False, pretrained_model=nuclei_DAPI_model_path)
        # n_model = models.CellposeModel(gpu=False, model_type = 'nuclei')
        n_channels = [3,0]
        n_masks, n_flows, n_styles = n_model.eval(imgDAPI, diameter=15, channels=n_channels)



        # 4 type cell cycle
        df_overlay=overlay_mask(dapi_mask=n_masks,cellmask=masks)
        df_cell_cycle=measure.regionprops_table(n_masks, img, properties=Feature_list )
        df_cell_cycle=pd.DataFrame(df_cell_cycle)
        df_cell_cycle.columns=['Nuclei_ID','area', 'max_intensity_DAPI', 'max_intensity_Tub', 'max_intensity_Edu','max_intensity_H3','mean_intensity_DAPI','mean_intensity_Tub','mean_intensity_Edu', 'mean_intensity_H3']

        df_result = pd.merge(df_cell_cycle, df_overlay, how="outer", on=["Nuclei_ID"])
        df_result['integrated_int_DAPI']=df_result['mean_intensity_DAPI']*df_result['area']

        df_gn = dna_norm(df_result)
        df_result['Image_ID']= image.getName

        # df_cell_phase=pd.concat([df_gn,df_cell_phase])
        # #
        #
        G1_list=g1_list(df_gn)
        # G1_list = [x for x in G1_list if np.isnan(x) == False]
        S_list=s_list(df_gn)
        # S_list = [x for x in S_list if np.isnan(x) == False]
        G2_list=g2_list(df_gn)
        # G2_list = [x for x in G2_list if np.isnan(x) == False]
        M_list=m_list(df_gn)
        # M_list = [x for x in M_list if np.isnan(x) == False]

        cell_list_g1=Cell_list(df_gn,G1_list)

        cell_list_s=Cell_list(df_gn,S_list)
        cell_list_g2=Cell_list(df_gn,G2_list)
        cell_list_m=Cell_list(df_gn,M_list)

        # #
        save_data_label_g1(list_nuclei=G1_list,list_cyto=cell_list_g1,ID=id_img)
        save_data_label_s(list_nuclei=S_list,list_cyto=cell_list_s,ID=id_img)
        save_data_label_g2(list_nuclei=G2_list,list_cyto=cell_list_g2,ID=id_img)
        save_data_label_m(list_nuclei=M_list,list_cyto=cell_list_m,ID=id_img)

#
#         df_label=pd.concat([df_g1,df_s,df_g2,df_M,df_result])
#
#
# df_label.head()


0
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
1
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
2
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
3
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
4
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
5
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
6
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 1080, 4)
(1080, 108